In [1]:
import pandas as pd
import Clean
import numpy as np

In [2]:
offers = pd.read_csv("offer_acceptance_offers.csv",low_memory=False)
orders = pd.read_csv('offer_acceptance_orders.csv',low_memory=False)
orders = Clean.clean_orders(orders)
offers = Clean.clean_offers(offers)

In [3]:
# create FTL Orders and Joined DataFrame with FTL Order to Offers
ftl = orders[orders['TRANSPORT_MODE'] == 'FTL']
join = ftl.set_index('REFERENCE_NUMBER').join(offers.set_index('REFERENCE_NUMBER'),how = 'inner')

In [4]:
# making sure all offers are quote
print(join[join['OFFER_TYPE'] == 'quote'].shape)
print(join.shape)

(269354, 30)
(269354, 30)


In [7]:
# create data set with only FTL offers and get a list of unique FTL Offers
ftlOffers = join[['CARRIER_ID','CREATED_ON_HQ','RATE_USD']].sort_values('CREATED_ON_HQ').head(100000)
offersRefNumList = np.unique(ftlOffers.index)
len(offersRefNumList)

34158

In [8]:
refNums = []
carrierIDs = []
rates = []
# loop through unique ref numbers
for offerRef in offersRefNumList:
    # get the order that goes to a refrence number
    order = ftl.loc[ftl['REFERENCE_NUMBER'] == offerRef]
    # get the offer as an array
    offer = ftlOffers.loc[offerRef].values.tolist()
    cost = order['ESTIMATED_COST_AT_ORDER'].reset_index()['ESTIMATED_COST_AT_ORDER'][0] * 1.5
    if isinstance(offer[0],str):
        if cost > offer[2]:
            refNums.append(offerRef)
            carrierIDs.append(offer[0])
            rates.append(offer[2])
    else:
        for ind in offer:
            if cost > ind[2]:
                refNums.append(offerRef)
                carrierIDs.append(ind[0])
                rates.append(ind[2])
                break          

In [9]:
len(refNums)

33551

In [18]:
flockRate = join[join['LOAD_DELIVERED_FROM_OFFER']]['RATE_USD']

In [23]:
results = pd.DataFrame({'refNums':refNums,'Carriers':carrierIDs,'Rate':rates}).set_index('refNums')
results['FlockRate'] = flockRate

In [31]:
results['Difference'] = results['Rate'] - results['FlockRate']
results['Difference'].sum()

2365557.0048

In [32]:
len(results['Rate'])

33551

In [35]:
len(results['FlockRate'].dropna())

30780